In [1]:
import tensorflow as tf;

2023-09-14 22:13:10.161031: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-14 22:13:10.749493: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-14 22:13:10.752696: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 22:13:12.431943: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class ConvNet:
    """ this class approximately implements the neural network demonstrated in the article
      "Deep learning-based Phase Retrieval Scheme for Minimum-phase Signal Recovery"
    """
    def __init__(self, samples: int, filters: int, kernel_size: int, strides: int):
        """ Initializes the class with the main parameters of the convolutional network
        Args:
            samples     (int): signal strength samples
            filters     (int): number of convolution layer filters
            kernel_size (int): number of kernels in the convolution layer
            strides     (int): number of steps (strides) in the convolution layer
        """
        self.samples = 128
        self.filters = 32
        self.kernel_size = 3
        self.strides = 2
    
    def skipConnection(self, input_layer):
        """ method that implements the skip layer
        Args:
            input_layer (Tensor): receives the tensor of any layer
        Returns:
            Tensor: returns the jump layer
        """
        return tf.keras.layers.Conv1D(filters=self.filters, kernel_size=1, activation='linear')(input_layer)

    def downsampling(self, input_layer):
        """ method that implements downsampling simulation
        Args:
            input_layer (Tensor): receives the tensor of any layer
        Returns:
            Tensor: sum of tensors between the parallel layer and the second convolution
        """
        Dtower_0 = tf.keras.layers.Conv1D(self.filters, kernel_size=self.kernel_size, padding='same', strides=self.strides, activation='relu')(input_layer)
        # parallel layer
        parallel = tf.keras.layers.Conv1D(self.filters, kernel_size=self.kernel_size, padding='same', strides=self.strides, activation='relu')(input_layer)
        Dtower_1 = tf.keras.layers.Conv1D(self.filters, kernel_size=3, padding='same', activation='relu')(Dtower_0)
        added = tf.keras.layers.Add()([Dtower_1, parallel])
        return added
    
    def upsampling(self, input_layer, skipConnection_layer):
        """ method that implements the simulation of upsampling operations
        Args:
            input_layer (Tensor): receives the tensor of any layer
            skipConnection_layer (Tensor): receives any jump layer
        Returns:
            Tensor: sum of the tensors between the parallel layer, the second convolution and the jump layer
        """
        tower_0 = tf.keras.layers.Conv1DTranspose(self.filters, kernel_size=self.kernel_size, padding='same', strides=self.strides, activation='relu')(input_layer)
        # parallel layer
        parallel = tf.keras.layers.Conv1DTranspose(self.filters, kernel_size=self.kernel_size, padding='same', strides=self.strides, activation='relu')(input_layer)
        tower_1 = tf.keras.layers.Conv1D(self.filters, kernel_size=self.kernel_size, padding='same', activation='relu')(tower_0)
        added = tf.keras.layers.Add()([tower_1, parallel, skipConnection_layer])
        return added
    
    def get_output_layer(self, input_layer):
        """ implements the convolutional model output layer
        Args:
            input_layer (Tensor): receives the tensor of any layer
        Returns:
            Tensor: returns the last layer of the convolutional model in a modular way
        """
        output_layer = tf.keras.layers.Conv1D(2, kernel_size=self.kernel_size, padding='same', activation='linear')(input_layer)
        flatten = tf.keras.layers.Flatten()(output_layer)
        output = tf.keras.layers.Dense(2, name='output', activation='linear')(flatten)
        return output
    
    def get_fist_layer(self):
        """ Implements neural network input and normalization form
        Returns:
            Tensor: returns the input format
            Tensor: returns a ReLU activation layer
        """
        model = tf.keras.Sequential()
        input_shape = tf.keras.Input(shape=(self.samples, 1))
        
        normalize = tf.keras.layers.BatchNormalization()(input_shape)
        relu_layer = tf.keras.layers.Activation('relu')(normalize)
        return input_shape, relu_layer

## Example implementation for a depth three model

In [3]:
# defines the main parameters of the network
ConvModel = ConvNet(samples=128, filters=32, kernel_size=3, strides=2)
# defines the neural network
input_shape, relu_layer = ConvModel.get_fist_layer()
# defines the downsampling operation
layer_D0 = ConvModel.downsampling(relu_layer)
layer_D1 = ConvModel.downsampling(layer_D0)
layer_D3 = ConvModel.downsampling(layer_D1)
# defines the upsampling operation
layer_U0 = ConvModel.upsampling(layer_D3, ConvModel.skipConnection(layer_D1))
layer_U1 = ConvModel.upsampling(layer_U0, ConvModel.skipConnection(layer_D0))
layer_U2 = ConvModel.upsampling(layer_U1, ConvModel.skipConnection(relu_layer))
# defines the model output
outputModel = ConvModel.get_output_layer(layer_U2)
# get your model parameters
model = tf.keras.Model(input_shape, outputModel)
model.summary()

2023-09-14 22:13:14.770333: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-14 22:13:14.770732: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 1)]             0         []                            
                                                                                                  
 batch_normalization (Batch  (None, 128, 1)               4         ['input_1[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 128, 1)               0         ['batch_normalization[0][0]'] 
                                                                                                  
 conv1d (Conv1D)             (None, 64, 32)               128       ['activation[0][0]']      